# Downstream: Predicting Stimuli Type from Scanpath

- Author: Beibin Li
- Date: Sept/20/2021
- Refer to Section 4.1 of [arxiv/2108.05025](https://arxiv.org/abs/2108.05025)


Here, we perform naive k-shot n-way supervised learning method.

Note that the FixaTon dataset belongs to MIT, and you need to download this MIT1003 dataset from [here](http://people.csail.mit.edu/tjudd/WherePeopleLook/index.html).

Then, you can use the sample_data/preprocess_FixaTons.py code to pre-process the scanpath data to a compatible format.
You can check "sample_data/FixaTons/MIT1003/clean_data" to see some example cleaned scanpaths.


## Setup Parameters

In [ ]:
#@title Import OBF libraries
from obf.model import ae
from obf.model import creator
from obf.dataloader.augmenter import train_transform, valid_transform
from obf.dataloader.mit_loader import MITDataset

from obf.utils.metrics import top_k_accuracy

In [ ]:
#@title Import torch, numpy, scipy, and other libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix

import numpy as np
import pandas as pd

import tqdm
import termcolor

import os
import glob
import datetime


Here, we will setup the path and training parameters.

The MIT1003 dataset should be downloaded and cleaned in the `INPUT_DATA_DIR` folder.
The pre-trained model should be saved in the `PRE_TRAIN_DIR` folder.

We perform `WAYS`-classificaiton with `WAYS` classes, and we have `SHOTS` examples (scanpaths) for each of the class.

The `BATCH_SIZE`, `LEARNING_RATE`, and `EPOCHS` are arbitrary settings for the deep learning experiment.

In [ ]:
#@Parameters Settings
INPUT_DATA_DIR = "../gaze_data/FixaTons/MIT1003/clean_data/"  #@param
PRE_TRAIN_DIR = "pre_weights/sample_weights/"  #@param
OUTPUT_DIR = "downstream_cache/"  #@param

SHOTS = 5  #@param
WAYS = 100  #@param

BATCH_SIZE = 4  #@param
LEARNING_RATE = 0.001  #@param
EPOCHS = 100  #@param
REPORT_INTERVAL = 10  #@param

USE_CUDA = torch.cuda.is_available() 

In [ ]:
# TRAIN_TYPE should either be "freeze" or "tune",
# if "freeze", we will freeze the Conv/RNN pre-trained encoder, and only fine-tune the new FC layers in the classifier
# if "tune", we will fine-tune the whole model.

TRAIN_TYPE = "tune"  #@param


In [ ]:
#@title Create and setup output path
time_str = datetime.datetime.now().strftime("%Y_%m_%d_%H.%M.%S")

checkpt_name = os.path.join(OUTPUT_DIR, time_str + ".txt")
log_dirname = os.path.join(OUTPUT_DIR, time_str + "_log")
model_save_path = os.path.join(log_dirname, "model.pt")


os.makedirs(log_dirname, exist_ok=True)


# Tensorboard writer
summary_writer = SummaryWriter(log_dir = log_dirname, flush_secs = 10)

## Create Model from Pre-Trained Weights

In [ ]:
#@title We can load the pre-trained encoder first
encoder = creator.load_encoder(PRE_TRAIN_DIR, use_cuda=USE_CUDA)

In [ ]:
#@title Create a classifier model, which concatenated FC layers after the encoder.
model = creator.create_classifier_from_encoder(encoder, hidden_layers=[256, 512], n_output=WAYS, dropout=0.5)


In [ ]:
creator.print_models_info(["original encoder", "current model"], [encoder, model])

In [ ]:
if USE_CUDA:
    model = model.cuda()

## Get Data Loader

In [ ]:
 
train_ds = MITDataset(mode="train", folder_name=INPUT_DATA_DIR, aug_transform=train_transform, shot=SHOTS, way=WAYS)
valid_ds = MITDataset(mode="valid", folder_name=INPUT_DATA_DIR, aug_transform=valid_transform, shot=SHOTS, way=WAYS)
test_ds = MITDataset(mode="test", folder_name=INPUT_DATA_DIR, aug_transform=valid_transform, shot=SHOTS, way=WAYS)


In [ ]:
train_loader  = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, 
                       shuffle=True, pin_memory=True, num_workers=0)  
 
valid_loader  = torch.utils.data.DataLoader(valid_ds, batch_size=BATCH_SIZE, 
                       shuffle=False, pin_memory=True, num_workers=0)

test_loader  = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, 
                       shuffle=False, pin_memory=True, num_workers=0)

print("Data Loader Set")

## Training

In [ ]:
def run_model(mode, model, dataloader, writer, epoch_id, optimizer=None):
  # mode is either "train" or "test" or "valid"
  assert( epoch_id is not None )

  criterion = nn.CrossEntropyLoss()

  if mode == "train": 
    model = model.train( True )
  else:
    model.eval()
    
  epoch_losses = []
  
  reals = []
  preds = []
  probs = []
  for signal, label in dataloader:
    signal = signal.float()
    label = label.long().reshape(-1)
    # pdb.set_trace()
    if USE_CUDA:  
      signal = signal.cuda()
      label = label.cuda()

    # pdb.set_trace()

    if mode == "train": 
      optimizer.zero_grad()

      if signal.shape[0] < 2:
        continue # batch norm needs more than 1 sample
 
    # forward + backward + optimize
    outputs = model(signal)
    loss = criterion(outputs, label)
    epoch_losses.append(loss.item())

    reals += label.cpu().numpy().tolist()
    preds += torch.argmax(outputs, dim=1).detach().cpu().numpy().tolist()
    probs.append(outputs.detach())

    if mode == "train":
      loss.backward()
      optimizer.step()

  total_loss = np.nanmean(epoch_losses)
  acc = np.sum(np.array(reals) == np.array(preds)) / len(reals)
  
  probs = torch.cat(probs, dim=0)
  top_5_acc = top_k_accuracy(probs.detach().cpu().numpy(), reals, k=5)
  
  f1 = f1_score(reals, preds, average="weighted")
  # auc = roc_auc_score(reals, probs, average="weighted")

  
  writer.add_scalar(mode + "/loss", total_loss, epoch_id)
  writer.add_scalar(mode + "/acc", acc, epoch_id)
  writer.add_scalar(mode + "/top_5_acc", top_5_acc, epoch_id)
  writer.add_scalar(mode + "/f1", f1, epoch_id)
  # writer.add_scalar(mode + "/auc", auc, epoch_id)
  writer.file_writer.flush()
  
  # print("#" * 50)
  if epoch_id % REPORT_INTERVAL == 0 or epoch_id == EPOCHS - 1:
    msg = "#" * 5 + "%s, Epoch: %d, Accuracy: %.2f, F-1: %.2f, Top-5: %.2f; Loss: %.2f" % (mode, 
             epoch_id, acc, f1, top_5_acc, total_loss)

    if mode == "train":
      color = "red"
    elif mode == "valid":
      color = "yellow"
    else:
      color = "green"

    print(termcolor.colored(msg, color=color))
 
    # print("reals", reals, "preds", preds)
    # print(confusion_matrix(reals, preds))
  
  return acc

In [ ]:
#@title Setup the optimizer based on "freeze" or "tune"

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

if TRAIN_TYPE == "freeze":
  optimizer = optim.Adam(model[1:].parameters(), lr=LEARNING_RATE) # Freeze the Encoder
  print(termcolor.colored("We will FREEZE the encoder.", "blue"))
elif TRAIN_TYPE == "tune":
  print(termcolor.colored("We will TUNE the WHOLE model.", "blue"))
else:
  raise "Unknown mode. It should be one of (tune, freeze, new)"  

In [ ]:
# %% Begin Training 
train_accs = []
valid_accs = []
test_accs = []

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=max(1, int(EPOCHS / 5)), gamma=0.5)

for epoch in tqdm.tqdm(range(EPOCHS)):
  auc = run_model("train", model, train_loader, summary_writer, epoch_id = epoch, optimizer=optimizer)
  summary_writer.add_scalar( "train/learning_rate", scheduler.get_last_lr()[-1], epoch )

  scheduler.step()

  torch.save(model, model_save_path)

  with torch.no_grad():
     valid_acc = run_model("valid", model, valid_loader, summary_writer, epoch_id=epoch)
     test_acc = run_model("test", model, test_loader, summary_writer, epoch_id=epoch)
     train_accs.append(auc)
     valid_accs.append(valid_acc)
     test_accs.append(test_acc)


times = range(len(train_accs))
plt.plot(times, train_accs, color="blue", label="train", alpha=0.5)
plt.plot(times, valid_accs, color="yellow", label="valid", alpha=0.5)
plt.plot(times, test_accs, color="red", label="test", alpha=0.5)
plt.legend()
plt.show()



